## Thomas–Fermi main2_0 notebook

- Uses `solvers/solver4.py` (external potential from data)
- Configure either a list of files or a combined file with `DESIRED_PAIRS`
- Supports multiple `XC_SCALES` and saves outputs to `analysis_folder/...`

In [ ]:
from pathlib import Path
from datetime import datetime

import numpy as np
import matplotlib.pyplot as plt

from solvers.solver4 import SimulationConfig, ThomasFermiSolver



In [ ]:
# Parameters
POTENTIAL_MODE = "files"  # or "combined"
POTENTIAL_FILES = [
    "data/0-data/VNS=0.5.txt",
]
COMBINED_FILE = "data/1-data/James2.txt"
DESIRED_PAIRS = []  # e.g., [(-4.0, -1.50), (-3.7, -1.50)]
XC_SCALES = [1.8]

GRID_N = 64
B_FIELD_T = 13.0
D_T = 30.0
D_B = 30.0
POTENTIAL_SCALE = 1.0
POTENTIAL_OFFSET = 0.0
MATRYOSHKA = False



In [ ]:
# Build and run a single case

def run_single_file(potential_file: str, xc_scale: float, out_dir: Path):
    cfg = SimulationConfig(
        Nx=GRID_N, Ny=GRID_N, B=B_FIELD_T,
        dt=D_T*1e-9, db=D_B*1e-9,
        potential_file=potential_file,
        potential_scale=POTENTIAL_SCALE, potential_offset=POTENTIAL_OFFSET,
        exc_file="data/0-data/Exc_data_new.csv",
        solver_type="solver4",
        exc_scale=float(xc_scale),
        use_matryoshka=MATRYOSHKA,
    )
    solver = ThomasFermiSolver(cfg)
    solver.optimise()
    solver.plot_results(save_dir=str(out_dir), title_extra=f"file={Path(potential_file).name}, XC={xc_scale:.3f}", show=False)
    solver.save_results(out_dir)

base_dir = Path("analysis_folder")
today = datetime.now().strftime("%Y%m%d")
date_dir = base_dir / today
date_dir.mkdir(parents=True, exist_ok=True)

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
batch_dir = date_dir / (timestamp + "_MAIN2_0_NOTEBOOK")
batch_dir.mkdir(parents=True, exist_ok=True)

case0 = batch_dir / "case_00"
case0.mkdir(parents=True, exist_ok=True)
run_single_file(POTENTIAL_FILES[0], XC_SCALES[0], case0)
print(f"Saved to {case0}")
